In [1]:
import grequests
from gevent import monkey
monkey.patch_all()

import sqlite3
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as ptch

from search_analysis import search_analysis
import numpy as np
import pandas as pd
import validators
import math
from pyvis.network import Network

from bs4 import BeautifulSoup, SoupStrainer

import requests
from collections import deque
import time
import re
from Search import Search

import sys
import logging
from langdetect import detect_langs
from requests_html import HTMLSession

# from Search import Search

%load_ext autoreload
%autoreload 2

C:\Users\Sam\AppData\Local\Temp\ipykernel_7904\1448681830.py:3: MonkeyPatchWarning: Patching more than once will result in the union of all True parameters being patched
  monkey.patch_all()
d:\NonSytemInfo\Python\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [2]:
# search_obj : Search = pd.read_pickle("dec_2022_test_2.pickle")
search_obj : Search = pd.read_pickle("dec_test_3.pickle")


# error_df = pd.read_pickle("2022-38473_search_searches_with_error.pickle")
# rejected = pd.read_pickle("2022-38473_search_rejected.pickle")

d:\NonSytemInfo\Python\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [21]:
Search.get_domain("https://en.wikipedia.org/wiki/Anseriformes")

'en.wikipedia.org'

In [9]:
len(search_obj.rejected)

9156

d:\NonSytemInfo\Python\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [8]:
eep = {i for i in search_obj.searches_with_error}
eep - set(search_obj.get_edge_list(reduced=True)["link"].to_list());

d:\NonSytemInfo\Python\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [22]:
len(search_obj.rejected)

49704

d:\NonSytemInfo\Python\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [ ]:
# grab a sets search results
con = sqlite3.connect("cve_july.db")


a = set()
for i in search_analysis.cve_sets["dec_2022_1"]:
    a = set.union(a, pd.read_sql(f"SELECT DISTINCT url FROM search_results WHERE cve = '{i}'", con)['url'].to_list())

con.close()


In [5]:
edges = search_obj.get_edge_list(True)
edges["domain_url"] = edges["url"].map(search_analysis.domain_scraper, na_action="ignore")
edges["domain_link"] = edges["link"].map(search_analysis.domain_scraper, na_action="ignore")
reduced = edges[["domain_url", "domain_link"]].drop_duplicates()
reduced.shape


(732, 2)

d:\NonSytemInfo\Python\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [29]:

g : nx.DiGraph = nx.from_pandas_edgelist(single_obj.get_edge_list(True), source="url", target="link")
net = Network(directed=True, bgcolor="#3d3d3d", font_color="white", select_menu=True)
net.from_nx(g)
net.force_atlas_2based()
net.show("single_obj.html")

# g : nx.DiGraph = nx.from_pandas_edgelist(toy_set.get_restricted_edge_set('2022-38473')[0], source="url", target="link")
# net = Network(directed=True, bgcolor="#3d3d3d", font_color="white", select_menu=True)
# net.from_nx(g)
# net.force_atlas_2based()
# net.show("regular.html")

d:\NonSytemInfo\Python\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [2]:
a = pd.read_pickle("dec_2022_1_to_search.pickle")

d:\NonSytemInfo\Python\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [18]:
tourble_makers = [a.pop() for i in range(500)]

In [11]:
session = requests.Session()
headers = {
    "User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"
}

url = "https://en.wikipedia.org/wiki/Brant_(goose)"#"https://vuldb.com/?id.217002"
blocked_url_prefixes = {'packetstormsecurity.com/files/date',
                        'packetstormsecurity.com/files/author/',
                        'packetstormsecurity.com/files/download/',
                        'isc.sans.edu/handler_list/',
                        'cassandra.cerias.purdue.edu/CVE_changes',  # contains db of all CVE changes by month
                        'cve.mitre.org/cgi-bin/cvekey.cgi?keyword=',
                        'cve.mitre.org/data/refs/',
                        'nvd.nist.gov/vuln/full-listing/',
                        'nvd.nist.gov/vuln/search/results',
                        'krebsonsecurity.com/tag/',
                        'www.askwoody.com/forums/users/',
                        'www.askwoody.com/author/'}

start = time.time()
request = session.get(url=url, headers=headers, timeout=5, stream=False)
# print(request.text)
request_time = time.time() - start
# terms_found = Search.scrape([r"Mathematical Association of America"], request.text)
soup = BeautifulSoup(request.text, 'lxml', parse_only=SoupStrainer(attrs={"class":"vector-body"}, multi_valued_attributes=None))
print(soup.prettify())
nice_soup = soup.find_all('a', href=True)

print(re.findall(r"Mathematical Association of America", request.text, re.IGNORECASE))
# print(Search.search(restricted_domain=["en.wikipedia.org"], soup=soup, parent="en.wikipedia.org"))
dirty_links = list([i.get('href') for i in nice_soup])
absolutly_dirty_links = Search.gen_absolute_links(dirty_links, parent_url="https://en.wikipedia.org/wiki/Brant_(goose)")
real_links = list(filter(lambda url: validators.url(url), absolutly_dirty_links))
print(real_links)

# grabbed = None
# for i in dirty_links:
#     if str.__contains__(i,"Anseriformes"):
#         grabbed = i
# abs = Search.gen_absolute_link(grabbed, "en.wikipedia.org")
# print(validators.url(abs))
domain_filter = lambda x: Search.get_domain(x) in ["en.wikipedia.org"]
final_links = filter(domain_filter, real_links)
print(len(list(final_links)))


soup = BeautifulSoup(request.text, "html.parser", parse_only=SoupStrainer('a'))

lang = BeautifulSoup(request.text, "html.parser", parse_only=SoupStrainer('html')).find("html").get("lang")
# print(soup)

# soup = BeautifulSoup(request.text, 'html.parser')
# found_urls = Search.search(soup, url)
# print(found_urls)
# # print(request.text.__sizeof__())

# print(f"total time: {request_time}, \n headers: {request.headers}\n")
# print(list(request.headers.keys()))
# request.close()




    

<!DOCTYPE html>
<div aria-labelledby="firstHeading" class="vector-body" data-mw-ve-target-container="" id="bodyContent">
 <div class="vector-body-before-content">
  <div class="mw-indicators">
  </div>
  <div class="noprint" id="siteSub">
   From Wikipedia, the free encyclopedia
  </div>
 </div>
 <div id="contentSub">
  <div id="mw-content-subtitle">
  </div>
 </div>
 <div class="mw-body-content" id="mw-content-text">
  <div class="mw-content-ltr mw-parser-output" dir="ltr" lang="en">
   <div class="shortdescription nomobile noexcerpt noprint searchaux" style="display:none">
    Species of bird
   </div>
   <style data-mw-deduplicate="TemplateStyles:r1033289096">
    .mw-parser-output .hatnote{font-style:italic}.mw-parser-output div.hatnote{padding-left:1.6em;margin-bottom:0.5em}.mw-parser-output .hatnote i{font-style:normal}.mw-parser-output .hatnote+link+.hatnote{margin-top:-0.5em}
   </style>
   <div class="hatnote navigation-not-searchable" role="note">
    "Black Brant" redirects 

d:\NonSytemInfo\Python\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [3]:

headers = {
    "User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"
}


blocked_url_prefixes = {'packetstormsecurity.com/files/date',
                        'packetstormsecurity.com/files/author/',
                        'packetstormsecurity.com/files/download/',
                        'isc.sans.edu/handler_list/',
                        'cassandra.cerias.purdue.edu/CVE_changes',  # contains db of all CVE changes by month
                        'cve.mitre.org/cgi-bin/cvekey.cgi?keyword=',
                        'cve.mitre.org/data/refs/',
                        'nvd.nist.gov/vuln/full-listing/',
                        'nvd.nist.gov/vuln/search/results',
                        'krebsonsecurity.com/tag/',
                        'www.askwoody.com/forums/users/',
                        'www.askwoody.com/author/'}

print(len(search_obj.rejected))
i = 0
responce_failed = 0
too_big = 0
bad_type = 0
no_cves = 0
too_many_links = 0
lang_error = 0
for url in search_obj.rejected:


    i += 1
    if i % 50 == 0:
        print(f"percent: {str(i/len(search_obj.rejected))[0:5]}%, responce_failed = {responce_failed}, too_big = {too_big}, bad_type = {bad_type}, no_cves = {no_cves}, too_many_links = {too_many_links}, lang_error = {lang_error}")

    try:
        responce = requests.get(url=url, headers=headers, timeout=10, stream=True)
        

        if responce == None:
            responce_failed += 1
            continue
        content_length = responce.headers.get("Content-Length")
        if content_length != None and content_length.isdigit() and int(content_length) > 10000000:
            responce.close()
            too_big += 1
            continue
        doc_type = responce.headers.get("Content-Type")
        if doc_type != None and doc_type == "unknown/unknown":
            responce.close()
            bad_type += 1
            continue
        cves_found = set.intersection(Search.scrape(responce.text), {'2022-38473'})
        if len(cves_found) == 0:
            responce.close()
            no_cves += 1
            continue

        soup = BeautifulSoup(responce.text, 'lxml', parse_only=SoupStrainer(['a', 'html']))
        found_urls = Search.search(soup, url)
        absolute = Search.gen_absolute_links(found_urls, parent_url=url)
        if(len(absolute) > 500): # add to rejected set any elements that have over a threshold of links
            responce.close()
            too_many_links += 1
            continue

        lang = soup.find("html").get("lang")
        if lang != None:
            if lang != "en":
                responce.close()
                lang_error += 1
                continue
        else: 
            english = Search.is_english(soup.text)
            if not english:
                responce.close()
                lang_error += 1
                continue
        
        print("stinky url: " + url + ", found: " + str(cves_found))



    except Exception as e:
        print("url failed: ", str(e), url)
        if responce is not None:
            responce.close()
        continue

    
    responce.close()
        


    

63011
percent: 0.000%, responce_failed = 0, too_big = 0, bad_type = 0, no_cves = 49, too_many_links = 0, lang_error = 0
percent: 0.001%, responce_failed = 0, too_big = 0, bad_type = 0, no_cves = 99, too_many_links = 0, lang_error = 0
percent: 0.002%, responce_failed = 0, too_big = 0, bad_type = 0, no_cves = 149, too_many_links = 0, lang_error = 0
percent: 0.003%, responce_failed = 0, too_big = 0, bad_type = 0, no_cves = 199, too_many_links = 0, lang_error = 0
percent: 0.003%, responce_failed = 0, too_big = 0, bad_type = 0, no_cves = 248, too_many_links = 0, lang_error = 1
percent: 0.004%, responce_failed = 0, too_big = 1, bad_type = 0, no_cves = 297, too_many_links = 0, lang_error = 1
percent: 0.005%, responce_failed = 0, too_big = 1, bad_type = 0, no_cves = 347, too_many_links = 0, lang_error = 1
percent: 0.006%, responce_failed = 0, too_big = 1, bad_type = 0, no_cves = 397, too_many_links = 0, lang_error = 1
percent: 0.007%, responce_failed = 0, too_big = 1, bad_type = 0, no_cves = 4

Traceback (most recent call last):
  File "d:\NonSytemInfo\Python\lib\site-packages\gevent\libuv\loop.py", line 44, in python_queue_callback
    def python_queue_callback(self, watcher_ptr, revents):
KeyboardInterrupt


percent: 0.122%, responce_failed = 0, too_big = 33, bad_type = 0, no_cves = 7648, too_many_links = 0, lang_error = 16


In [ ]:
"2022-40957" in search_analysis.cve_sets["dec_2022_1"]

False

In [16]:
url = 'https://www.cisa.gov/news-events/bulletins/sb22-361'
headers = {
    "User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"
}
responce = requests.get(url=url, headers=headers, timeout=10, stream=True)
if responce is None:
    print("bad responce: " + url)
if "Content-Length" in responce.headers.keys():
    content_length = responce.headers["Content-Length"]
    print(content_length.isdigit())
    if not content_length.isdigit() or int(content_length) > 5000000:
        responce.close()
        print("too big xd: " + url)


True


2898

d:\NonSytemInfo\Python\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [9]:
"https://www.cisa.gov/news-events/bulletins/sb22-361" in search_obj.get_edge_list()["link"].to_list()

False

d:\NonSytemInfo\Python\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [3]:
test_set = set([(1,2), (2,3), (3,4)])
test_set

{(1, 2), (2, 3), (3, 4)}

In [4]:
for item in test_set:
    print(item)
    if item[1] == 3:
        test_set.remove(item)
test_set

(2, 3)


RuntimeError: Set changed size during iteration

In [30]:
h = {"a": {1,2,3}}
print(type({1,2,3}))
h['a'] = set([1,2,2])

<class 'set'>


In [35]:
print(h.get("b"))


None


In [16]:
test_file : Search = pd.read_pickle("very_small_test.pickle")

d:\NonSytemInfo\Python\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [23]:
a = test_file.to_search
a.pop()

'https://en.wikipedia.org/w/index.php?title=Special:RecentChangesLinked&from=20231224233027&fromFormatted=23%3A30%2C+24+December+2023&hidemyself=1&hidenondamaging=1&days=30&target=Talk%3ABrant_goose'